## **1) Preprossessing**

### **Transformations for Customer Data table**

In [1]:
-- Check data types
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'WOODCORP_ETC_CUSTOMERDATA';

(5 rows affected)

Total execution time: 00:00:00.134

COLUMN_NAME,DATA_TYPE
CUSTOMER_NO,int
LOYALTY_PROGRAM,tinyint
AGE,tinyint
CUSTOMER_TYPE,tinyint
GENDER,tinyint


No need to chane anything:  
\- All in tinyint works given that all the columns in tinyint have values between 0 and 4.  
<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">- Int for the primary key (Cutomer_NO) is fine.</span>

In [2]:
-- Check if Primary Keys Exist
SELECT 
    column_name
FROM 
    information_schema.key_column_usage
WHERE 
    table_name = 'WOODCORP_ETC_CUSTOMERDATA' 
    AND constraint_name LIKE 'PK_%';

(0 rows affected)

Total execution time: 00:00:00.136

column_name


No primary key. Let's set it!

In [3]:
-- Drop existing primary key constraint if any
IF EXISTS (SELECT * FROM information_schema.table_constraints
           WHERE table_name = 'WOODCORP_ETC_CUSTOMERDATA'
           AND constraint_type = 'PRIMARY KEY')
BEGIN
    DECLARE @sql NVARCHAR(MAX)
    SET @sql = (SELECT 'ALTER TABLE WOODCORP_ETC_CUSTOMERDATA DROP CONSTRAINT ' + constraint_name
                FROM information_schema.table_constraints
                WHERE table_name = 'WOODCORP_ETC_CUSTOMERDATA'
                AND constraint_type = 'PRIMARY KEY')
    EXEC sp_executesql @sql
END

-- Add primary key constraint on CUSTOMER_NO
ALTER TABLE WOODCORP_ETC_CUSTOMERDATA
ADD CONSTRAINT WOODCORP_ETC_CUSTOMERDATA_PK PRIMARY KEY (CUSTOMER_NO);


: Msg 3725, Level 16, State 0, Line 1
The constraint 'WOODCORP_ETC_CUSTOMERDATA_PK' is being referenced by table 'WOODCORP_ETC_OFFLINE', foreign key constraint 'WOODCORP_ETC_OFFLINE_FK'.

: Msg 3727, Level 16, State 0, Line 1
Could not drop constraint. See previous errors.

Total execution time: 00:00:00.121

### **Transformations for Online Table**

In [60]:
-- Check data types
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'WOODCORP_ETC_ONLINE';

(6 rows affected)

Total execution time: 00:00:00.478

COLUMN_NAME,DATA_TYPE
ORDER_NO,int
CUSTOMER_NUMBER_ONLINE,int
ORDER_VALUE_ONLINE,float
DISCOUNT_VALUE_ONLINE,float
NUMBER_ITEMS_ONLINE,int
NUMBER_DISCOUNTED_ITEMS_ONLINE,int


All is good no need to change!

In [64]:
-- Check if Primary Keys Exist
SELECT 
    kcu.COLUMN_NAME
FROM 
    INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
    JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
        ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE 
    tc.TABLE_NAME = 'WOODCORP_ETC_ONLINE'
    AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY';

(0 rows affected)

Total execution time: 00:00:00.539

COLUMN_NAME


In [65]:
-- Check if foreign Keys Exist
SELECT 
    kcu.COLUMN_NAME
FROM 
    INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
    JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
        ON tc.CONSTRAINT_NAME = kcu.CONSTRAINT_NAME
WHERE 
    tc.TABLE_NAME = 'WOODCORP_ETC_ONLINE'
    AND tc.CONSTRAINT_TYPE = 'FOREIGN KEY';

(0 rows affected)

Total execution time: 00:00:00.504

COLUMN_NAME


No keys defined! Let's add them

Attempted to set primary keys but encountered issues due to existing primary key constraint. There are 600 unmatched foreign key values in the CUSTOMER\_NUMBER\_ONLINE column that do not match any value in the CUSTOMER\_NO column of the WOODCORP\_ETC\_CUSTOMERDATA table. This indicates there are orders with no corresponding customer information.Let's find and remove these unmatched records as they hinder our ability to perform accurate analysis.

In [69]:
-- Find unmatched foreign key values
SELECT DISTINCT CUSTOMER_NUMBER_ONLINE 
FROM WOODCORP_ETC_ONLINE
WHERE CUSTOMER_NUMBER_ONLINE NOT IN (SELECT CUSTOMER_NO FROM WOODCORP_ETC_CUSTOMERDATA);


(606 rows affected)

Total execution time: 00:00:00.444

CUSTOMER_NUMBER_ONLINE
53844486
53879355
53933388
53955729
53970765
53973495
54002796
54028599
54096585
54096627


Since only 600 out of 3000 are affected we go aheand and revode the cases form the online order table.

In [70]:
-- Delete rows with unmatched customer numbers
DELETE FROM WOODCORP_ETC_ONLINE
WHERE CUSTOMER_NUMBER_ONLINE IN (SELECT CUSTOMER_NUMBER_ONLINE 
                                 FROM WOODCORP_ETC_ONLINE
                                 WHERE CUSTOMER_NUMBER_ONLINE NOT IN (SELECT CUSTOMER_NO FROM WOODCORP_ETC_CUSTOMERDATA));


(1453 rows affected)

Total execution time: 00:00:00.809

In [71]:
-- Drop existing primary key constraint if any for WOODCORP_ETC_ONLINE
IF EXISTS (SELECT * FROM information_schema.table_constraints
           WHERE table_name = 'WOODCORP_ETC_ONLINE'
           AND constraint_type = 'PRIMARY KEY')
BEGIN
    DECLARE @sql1 NVARCHAR(MAX)
    SET @sql1 = (SELECT 'ALTER TABLE WOODCORP_ETC_ONLINE DROP CONSTRAINT ' + constraint_name
                FROM information_schema.table_constraints
                WHERE table_name = 'WOODCORP_ETC_ONLINE'
                AND constraint_type = 'PRIMARY KEY')
    EXEC sp_executesql @sql1
END

-- Add primary key constraint on ORDER_NO for WOODCORP_ETC_ONLINE
ALTER TABLE WOODCORP_ETC_ONLINE
ADD CONSTRAINT WOODCORP_ETC_ONLINE_PK PRIMARY KEY (ORDER_NO);

-- Drop existing foreign key constraint if any for WOODCORP_ETC_ONLINE
IF EXISTS (SELECT * FROM information_schema.table_constraints
           WHERE table_name = 'WOODCORP_ETC_ONLINE'
           AND constraint_type = 'FOREIGN KEY')
BEGIN
    DECLARE @sql2 NVARCHAR(MAX)
    SET @sql2 = (SELECT 'ALTER TABLE WOODCORP_ETC_ONLINE DROP CONSTRAINT ' + constraint_name
                FROM information_schema.table_constraints
                WHERE table_name = 'WOODCORP_ETC_ONLINE'
                AND constraint_type = 'FOREIGN KEY')
    EXEC sp_executesql @sql2
END

-- Add foreign key constraint on CUSTOMER_NUMBER_ONLINE for WOODCORP_ETC_ONLINE
ALTER TABLE WOODCORP_ETC_ONLINE
ADD CONSTRAINT WOODCORP_ETC_ONLINE_FK FOREIGN KEY (CUSTOMER_NUMBER_ONLINE)
REFERENCES WOODCORP_ETC_CUSTOMERDATA (CUSTOMER_NO);



Commands completed successfully.

Total execution time: 00:00:01.060

### **Transformations Offline Table**

In [72]:
-- Check data types
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'WOODCORP_ETC_OFFLINE';

(8 rows affected)

Total execution time: 00:00:00.543

COLUMN_NAME,DATA_TYPE
ORDER_NO,float
CUSTOMER_NUMBER_OFFLINE,int
ORDER_VALUE_OFFLINE,float
DISCOUNT_VALUE_OFFLINE,float
NUMBER_ITEMS_OFFLINE,smallint
NUMBER_DISCOUNT_ITEMS_OFFLINE,tinyint
HIGH_QUALITY_WOOD,tinyint
LOW_QUALITY_WOOD,tinyint


All is good:  
\- SmallInt for values above 255, TinyInt for values below 255, Float is fine.  
<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">- Just need to convert ORDER_NO to Int.</span>

In [80]:
-- Transform ORDER_NO to a smaller range
UPDATE dbo.WOODCORP_ETC_OFFLINE
SET ORDER_NO = ORDER_NO / 1000000000; -- Adjust the divisor based on the number of extraneous zeros

(134758 rows affected)

Total execution time: 00:00:01.441

In [85]:
-- Change the data type of ORDER_NO to INT NOT NULL (SQL server seems to find null values while there aren't)
ALTER TABLE dbo.WOODCORP_ETC_OFFLINE
ALTER COLUMN ORDER_NO INT NOT NULL;


Commands completed successfully.

Total execution time: 00:00:01.281

In [1]:
-- Identify duplicate ORDER_NO values as we seem to encounter this issue when creating the keys
SELECT ORDER_NO, COUNT(*)
FROM dbo.WOODCORP_ETC_OFFLINE
GROUP BY ORDER_NO
HAVING COUNT(*) > 1;


(8178 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:01.459

ORDER_NO,(No column name)
20200,71
293050,11
213750,24
20710,132
21830,32
217560,10
188520,9
264520,5
218240,4
22580,60


In [2]:
-- Add a surrogate key column
ALTER TABLE WOODCORP_ETC_OFFLINE ADD ID INT IDENTITY(1,1) PRIMARY KEY;


Commands completed successfully.

Total execution time: 00:00:04.581

A surrogate key is an artificial or synthetic key that is used to uniquely identify each record in a table. It is useful when there are no natural primary keys or when natural primary keys have issues such as duplicates or are too complex. By adding a surrogate key column (ID), we ensure each record has a unique identifier.

In [3]:
-- Drop existing primary key constraint if any
IF EXISTS (SELECT * FROM information_schema.table_constraints
           WHERE table_name = 'WOODCORP_ETC_OFFLINE'
           AND constraint_type = 'PRIMARY KEY')
BEGIN
    DECLARE @sql3 NVARCHAR(MAX)
    SET @sql3 = (SELECT 'ALTER TABLE WOODCORP_ETC_OFFLINE DROP CONSTRAINT ' + constraint_name
                FROM information_schema.table_constraints
                WHERE table_name = 'WOODCORP_ETC_OFFLINE'
                AND constraint_type = 'PRIMARY KEY')
    EXEC sp_executesql @sql3
END

-- Add primary key constraint on ID
ALTER TABLE WOODCORP_ETC_OFFLINE
ADD CONSTRAINT WOODCORP_ETC_OFFLINE_PK PRIMARY KEY (ID);

Commands completed successfully.

Total execution time: 00:00:01.235

In [6]:
-- Identify conflicting records
SELECT CUSTOMER_NUMBER_OFFLINE
FROM WOODCORP_ETC_OFFLINE
WHERE CUSTOMER_NUMBER_OFFLINE NOT IN (SELECT CUSTOMER_NO FROM WOODCORP_ETC_CUSTOMERDATA);


(57189 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:01.488

CUSTOMER_NUMBER_OFFLINE
70589919
54846504
58005978
83858295
60618204
62728416
54353511
87649491
67837128
64127529


In [7]:
-- Remove invalid records
DELETE FROM WOODCORP_ETC_OFFLINE
WHERE CUSTOMER_NUMBER_OFFLINE NOT IN (SELECT CUSTOMER_NO FROM WOODCORP_ETC_CUSTOMERDATA);


(57189 rows affected)

Total execution time: 00:00:01.113

Repeating the approach followed for the ONLINE dataset we removed the orders of customers not present in our customer database. (

In [8]:
-- Drop existing foreign key constraint if any
IF EXISTS (SELECT * FROM information_schema.table_constraints
           WHERE table_name = 'WOODCORP_ETC_OFFLINE'
           AND constraint_type = 'FOREIGN KEY')
BEGIN
    DECLARE @sql4 NVARCHAR(MAX)
    SET @sql4 = (SELECT 'ALTER TABLE WOODCORP_ETC_OFFLINE DROP CONSTRAINT ' + constraint_name
                FROM information_schema.table_constraints
                WHERE table_name = 'WOODCORP_ETC_OFFLINE'
                AND constraint_type = 'FOREIGN KEY')
    EXEC sp_executesql @sql4
END

-- Add foreign key constraint on CUSTOMER_NUMBER_OFFLINE
ALTER TABLE WOODCORP_ETC_OFFLINE
ADD CONSTRAINT WOODCORP_ETC_OFFLINE_FK FOREIGN KEY (CUSTOMER_NUMBER_OFFLINE)
REFERENCES WOODCORP_ETC_CUSTOMERDATA (CUSTOMER_NO);

Commands completed successfully.

Total execution time: 00:00:00.393

### **Null Values Handdling**

In [7]:
-- Check for null values in key columns
SELECT COUNT(*) AS NullCustomerNo
FROM dbo.WOODCORP_ETC_CUSTOMERDATA
WHERE CUSTOMER_NO IS NULL;

(1 row affected)

Total execution time: 00:00:00.394

NullCustomerNo
0


In [2]:

SELECT COUNT(*) AS NullOrderNoOnline
FROM dbo.WOODCORP_ETC_ONLINE
WHERE ORDER_NO IS NULL;


(1 row affected)

Total execution time: 00:00:00.465

NullOrderNoOnline
0


In [3]:
SELECT COUNT(*) AS NullCustomerNumberOnline
FROM dbo.WOODCORP_ETC_ONLINE
WHERE CUSTOMER_NUMBER_ONLINE IS NULL;

(1 row affected)

Total execution time: 00:00:00.315

NullCustomerNumberOnline
0


In [4]:
SELECT COUNT(*) AS NullOrderNoOffline
FROM dbo.WOODCORP_ETC_OFFLINE
WHERE ORDER_NO IS NULL;

(1 row affected)

Total execution time: 00:00:00.292

NullOrderNoOffline
0


In [5]:
SELECT COUNT(*) AS NullCustomerNumberOffline
FROM dbo.WOODCORP_ETC_OFFLINE
WHERE CUSTOMER_NUMBER_OFFLINE IS NULL;

(1 row affected)

Total execution time: 00:00:00.341

NullCustomerNumberOffline
0


There aren't any null values.

## **2) Exploratory Data Analysis**

### **Basic Questions**

In [11]:
-- 1) How many unique customers are there?
SELECT COUNT(DISTINCT CUSTOMER_NO) AS UniqueCustomers 
FROM dbo.WOODCORP_ETC_CUSTOMERDATA;


(1 row affected)

Total execution time: 00:00:00.395

UniqueCustomers
13221


In [4]:
-- 2) What is the average order value online?
SELECT ROUND(AVG(ORDER_VALUE_ONLINE), 0) AS AverageOrderValueOnline 
FROM dbo.WOODCORP_ETC_ONLINE;


(1 row affected)

Total execution time: 00:00:00.279

AverageOrderValueOnline
17219


In [10]:
-- Check for offline
SELECT ROUND(AVG(ORDER_VALUE_OFFLINE), 0) AS AverageOrderValueOnline 
FROM dbo.WOODCORP_ETC_OFFLINE;

(1 row affected)

Total execution time: 00:00:00.247

AverageOrderValueOnline
4180


WoodCorp's online orders total 835,606 items, while offline orders total 2,456,720 items. Additionally, the average value of online orders is $17,219, compared to $4,180 for offline orders. This suggests that while the volume of transactions is higher offline, the value per transaction is significantly higher online, indicating potential growth opportunities for high-value sales through online channels while leveraging the high volume of offline transactions.

In [15]:
-- 3) What is our most expensive order?
SELECT ROUND(MAX(ORDER_VALUE_ONLINE), 0) AS MostExpensiveOrder 
FROM dbo.WOODCORP_ETC_ONLINE;


(1 row affected)

Total execution time: 00:00:00.067

MostExpensiveOrder
208788


In [16]:
-- 4) Who (customer number) ordered our most expensive order?
SELECT CUSTOMER_NUMBER_ONLINE 
FROM dbo.WOODCORP_ETC_ONLINE 
WHERE ORDER_VALUE_ONLINE = (SELECT MAX(ORDER_VALUE_ONLINE) FROM dbo.WOODCORP_ETC_ONLINE);


(1 row affected)

Total execution time: 00:00:00.038

CUSTOMER_NUMBER_ONLINE
86040033


In [17]:
-- 5) How many customers are part of the loyalty program?
SELECT COUNT(*) AS LoyaltyProgramMembers 
FROM dbo.WOODCORP_ETC_CUSTOMERDATA 
WHERE LOYALTY_PROGRAM = 1;


(1 row affected)

Total execution time: 00:00:00.040

LoyaltyProgramMembers
4604


34.8% of the 13,221 unique customers are in the loyalty program. This is good but we could consider increasing the enrolment in the program with enhance benefits, targeted marketing, and referral incentives to boost loyalty program participation.

In [18]:
-- 6) How many orders include high quality wood?
SELECT COUNT(*) AS HighQualityOrders 
FROM dbo.WOODCORP_ETC_OFFLINE 
WHERE HIGH_QUALITY_WOOD = 1;


(1 row affected)

Total execution time: 00:00:00.241

HighQualityOrders
15368


Out of 77,569 offline orders, 15,368 include high-quality wood. This represents approximately 19.82% of all offline orders. This indicates a significant demand for high-quality wood, suggesting the company should continue to focus on and possibly expand its high-quality wood product offerings to meet customer demand. Of course also depending on what is the benefit made out of high quality versus low.

In [20]:
-- 7) What percentage of customers is older than 55 years?
SELECT ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM dbo.WOODCORP_ETC_CUSTOMERDATA)), 0) AS PercentageOlderThan55
FROM dbo.WOODCORP_ETC_CUSTOMERDATA 
WHERE AGE = 4;


(1 row affected)

Total execution time: 00:00:00.058

PercentageOlderThan55
15.000000000000


15% of WoodCorp's customers are older than 55 years, indicating they are underrepresented compared to an even distribution of 25%.  
Strategic Options:  
\- **Increase Efforts for Older Customers**: Target marketing and products to attract more older customers, leveraging their potential value.  
\- **Focus on Current Strengths**: Continue to engage and satisfy demographics with higher proportions to maximize returns.  

—\> Balanced approach.

In [5]:
-- 8) Of what customer type do we have the least customers apart from customer type 0?
SELECT TOP 1 
    CASE 
        WHEN CUSTOMER_TYPE = 1 THEN 'Transporter'
        WHEN CUSTOMER_TYPE = 2 THEN 'Factory'
        WHEN CUSTOMER_TYPE = 3 THEN 'Trader'
        WHEN CUSTOMER_TYPE = 4 THEN 'Private'
        ELSE 'Unknown'
    END AS CUSTOMER_TYPE,
    COUNT(*) AS Count
FROM dbo.WOODCORP_ETC_CUSTOMERDATA
WHERE CUSTOMER_TYPE <> 0
GROUP BY CUSTOMER_TYPE
ORDER BY Count ASC;


(1 row affected)

Total execution time: 00:00:00.042

CUSTOMER_TYPE,Count
Private,1026


To improve the market penetration and increase the customer base in the "Private" segment, WoodCorp should consider targeted marketing strategies and promotions. This might include personalized offers, loyalty programs, and tailored communications to attract more private customers.

In [25]:
-- 9) How many items have been ordered online?
SELECT SUM(NUMBER_ITEMS_ONLINE) AS TotalItemsOrderedOnline 
FROM dbo.WOODCORP_ETC_ONLINE;


(1 row affected)

Total execution time: 00:00:00.036

TotalItemsOrderedOnline
835606


WoodCorp's online orders total 835,606 items, while offline orders total 2,456,720 items. Additionally, the average value of online orders is $17,219, compared to $4,180 for offline orders. This suggests that while the volume of transactions is higher offline, the value per transaction is significantly higher online, indicating potential growth opportunities for high-value sales through online channels while leveraging the high volume of offline transactions.

### **Intermediate Questions**

In [26]:
-- 1) How many women are part of the loyalty program?
SELECT COUNT(*) AS WomenInLoyaltyProgram
FROM dbo.WOODCORP_ETC_CUSTOMERDATA
WHERE GENDER = 1  -- Female
AND LOYALTY_PROGRAM = 1;  -- In loyalty program



(1 row affected)

Total execution time: 00:00:00.050

WomenInLoyaltyProgram
1449


Additional investigations to see how much it represents and if men observe different proportions:

In [3]:
-- Calculate the ratio of women in the loyalty program
SELECT 
    (SELECT COUNT(*) 
     FROM dbo.WOODCORP_ETC_CUSTOMERDATA
     WHERE GENDER = 1 
     AND LOYALTY_PROGRAM = 1) * 1.0 / 
    (SELECT COUNT(*) 
     FROM dbo.WOODCORP_ETC_CUSTOMERDATA
     WHERE GENDER = 1) AS RatioWomenInLoyaltyProgram;


(1 row affected)

Total execution time: 00:00:00.038

RatioWomenInLoyaltyProgram
0.343364928909


In [4]:
-- Calculate the ratio of men in the loyalty program
SELECT 
    (SELECT COUNT(*) 
     FROM dbo.WOODCORP_ETC_CUSTOMERDATA
     WHERE GENDER = 0 
     AND LOYALTY_PROGRAM = 1) * 1.0 / 
    (SELECT COUNT(*) 
     FROM dbo.WOODCORP_ETC_CUSTOMERDATA
     WHERE GENDER = 0) AS RatioMenInLoyaltyProgram;


(1 row affected)

Total execution time: 00:00:00.053

RatioMenInLoyaltyProgram
0.350516609265


Out of the total female customers, approximately 34.34% are enrolled in the loyalty program. In comparison, around 35.05% of male customers are part of the loyalty program. This indicates a similar level of engagement with the loyalty program between male and female customers, suggesting that promotional efforts are reaching both demographics effectively and should only be accentuated but no change needed in the format.

In [6]:
-- 2) What customer type is relatively speaking responsible for most online orders?
SELECT 
    CASE 
        WHEN CUSTOMER_TYPE = 1 THEN 'Transporter'
        WHEN CUSTOMER_TYPE = 2 THEN 'Factory'
        WHEN CUSTOMER_TYPE = 3 THEN 'Trader'
        WHEN CUSTOMER_TYPE = 4 THEN 'Private'
        ELSE 'Unknown'
    END AS CUSTOMER_TYPE,
    COUNT(*) AS OrderCount
FROM dbo.WOODCORP_ETC_ONLINE AS O
JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
ON O.CUSTOMER_NUMBER_ONLINE = C.CUSTOMER_NO
GROUP BY CUSTOMER_TYPE
ORDER BY OrderCount DESC;


(5 rows affected)

Total execution time: 00:00:00.055

CUSTOMER_TYPE,OrderCount
Factory,2874
Transporter,2869
Trader,825
Private,348
Unknown,111


This insight suggests that efforts to optimize online sales processes should prioritize factories and transporters due to their high order volume.

In [7]:
-- 3) What customer type is on average the youngest?
SELECT 
    CASE 
        WHEN CUSTOMER_TYPE = 1 THEN 'Transporter'
        WHEN CUSTOMER_TYPE = 2 THEN 'Factory'
        WHEN CUSTOMER_TYPE = 3 THEN 'Trader'
        WHEN CUSTOMER_TYPE = 4 THEN 'Private'
        ELSE 'Unknown'
    END AS CUSTOMER_TYPE,
    ROUND(AVG(CAST(AGE AS FLOAT)), 5) AS AvgAge
FROM dbo.WOODCORP_ETC_CUSTOMERDATA
GROUP BY CUSTOMER_TYPE
ORDER BY AvgAge ASC;


(5 rows affected)

Total execution time: 00:00:00.043

CUSTOMER_TYPE,AvgAge
Unknown,"2,21632"
Private,"2,21637"
Trader,"2,21883"
Factory,"2,34597"
Transporter,"2,76401"


Understanding the age demographics can help in tailoring marketing strategies to different customer types, with a focus on engaging younger customers more effectively.

In [30]:
-- 4) What age group orders the most online items?
SELECT AGE_GROUP, SUM(NUMBER_ITEMS_ONLINE) AS TotalItemsOrdered
FROM (
    SELECT CASE
        WHEN AGE = 1 THEN '0-18'
        WHEN AGE = 2 THEN '19-35'
        WHEN AGE = 3 THEN '36-55'
        WHEN AGE = 4 THEN '55+'
        WHEN AGE = 0  THEN  'NA' END AS AGE_GROUP,
        NUMBER_ITEMS_ONLINE
    FROM dbo.WOODCORP_ETC_ONLINE AS O
    JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
    ON O.CUSTOMER_NUMBER_ONLINE = C.CUSTOMER_NO
) AS AgeGroupItems
GROUP BY AGE_GROUP
ORDER BY TotalItemsOrdered DESC;


(4 rows affected)

Total execution time: 00:00:00.071

AGE_GROUP,TotalItemsOrdered
19-35,423244
36-55,333158
55+,66528
0-18,12676


This insight suggests that marketing and sales strategies should focus on the 19-35 and 36-55 age groups to maximize online sales

In [32]:
-- 5) Is the value of an offline order on average higher when high-quality wood is sold?
SELECT
    CASE WHEN HIGH_QUALITY_WOOD > 0 THEN 'With High-Quality Wood' ELSE 'Without High-Quality Wood' END AS Quality,
    ROUND(AVG(ORDER_VALUE_OFFLINE), 0) AS AvgOrderValue
FROM dbo.WOODCORP_ETC_OFFLINE
GROUP BY CASE WHEN HIGH_QUALITY_WOOD > 0 THEN 'With High-Quality Wood' ELSE 'Without High-Quality Wood' END;


(2 rows affected)

Total execution time: 00:00:00.076

Quality,AvgOrderValue
With High-Quality Wood,7008
Without High-Quality Wood,3481


The average value of offline orders containing high-quality wood is $7,008, compared to $3,481 for those without. This indicates that high-quality wood significantly increases order value. Further analysis could determine if the higher order values translate to higher overall profitability.

In [36]:
-- 6) Is the average value of an online order higher when more than 5 items are bought in discount?
SELECT
    CASE WHEN NUMBER_DISCOUNTED_ITEMS_ONLINE > 5 THEN 'More than 5 Discounted Items' ELSE '5 or Less Discounted Items' END AS DiscountCategory,
    ROUND(AVG(ORDER_VALUE_ONLINE), 0) AS AvgOrderValue
FROM dbo.WOODCORP_ETC_ONLINE
GROUP BY CASE WHEN NUMBER_DISCOUNTED_ITEMS_ONLINE > 5 THEN 'More than 5 Discounted Items' ELSE '5 or Less Discounted Items' END;


(2 rows affected)

Total execution time: 00:00:00.064

DiscountCategory,AvgOrderValue
More than 5 Discounted Items,18572
5 or Less Discounted Items,13454


Orders with more than 5 discounted items have a higher average value of $18,572 compared to $13,454 for those with 5 or fewer discounted items. This suggests that offering bulk discounts can significantly increase the order value.

In [33]:
-- 7) How many customers placed more than 3 orders in total? Distinguish between online and offline.
SELECT
    CUSTOMER_NO,
    SUM(CASE WHEN Source = 'Online' THEN 1 ELSE 0 END) AS OnlineOrders,
    SUM(CASE WHEN Source = 'Offline' THEN 1 ELSE 0 END) AS OfflineOrders,
    SUM(OrderCount) AS TotalOrders
FROM (
    SELECT CUSTOMER_NUMBER_ONLINE AS CUSTOMER_NO, COUNT(*) AS OrderCount, 'Online' AS Source
    FROM dbo.WOODCORP_ETC_ONLINE
    GROUP BY CUSTOMER_NUMBER_ONLINE
    UNION ALL
    SELECT CUSTOMER_NUMBER_OFFLINE AS CUSTOMER_NO, COUNT(*) AS OrderCount, 'Offline' AS Source
    FROM dbo.WOODCORP_ETC_OFFLINE
    GROUP BY CUSTOMER_NUMBER_OFFLINE
) AS AllOrders
GROUP BY CUSTOMER_NO
HAVING SUM(OrderCount) > 3;


(8615 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.236

CUSTOMER_NO,OnlineOrders,OfflineOrders,TotalOrders
57689514,1,1,5
78510288,1,1,7
58986849,0,1,15
88641975,0,1,14
64614207,0,1,8
81059568,0,1,15
88951077,0,1,4
62647728,0,1,4
57990126,1,1,7
62232117,0,1,14


In [8]:
-- Summ onliners
SELECT COUNT(*) AS OnlineCustomers
FROM (
    SELECT CUSTOMER_NUMBER_ONLINE AS CUSTOMER_NO, COUNT(*) AS OrderCount
    FROM dbo.WOODCORP_ETC_ONLINE
    GROUP BY CUSTOMER_NUMBER_ONLINE
    HAVING COUNT(*) > 3
) AS OnlineOrders;


(1 row affected)

Total execution time: 00:00:00.064

OnlineCustomers
1017


In [9]:
-- Sum Offliners
SELECT COUNT(*) AS OfflineCustomers
FROM (
    SELECT CUSTOMER_NUMBER_OFFLINE AS CUSTOMER_NO, COUNT(*) AS OrderCount
    FROM dbo.WOODCORP_ETC_OFFLINE
    GROUP BY CUSTOMER_NUMBER_OFFLINE
    HAVING COUNT(*) > 3
) AS OfflineOrders;


(1 row affected)

Total execution time: 00:00:00.071

OfflineCustomers
7695


A much larger number of customers (7695) place more than three orders offline compared to online (1017). This indicates that while the online channel is utilized, the offline channel is the dominant mode of transaction for repeat customers. Strategic initiatives to boost online engagement could be beneficial to balance this distribution and leverage the digital sales channel more effectively.

In [7]:
-- 8) What customer type orders low-quality wood more often offline?
SELECT 
    CASE 
        WHEN CUSTOMER_TYPE = 1 THEN 'Transporter'
        WHEN CUSTOMER_TYPE = 2 THEN 'Factory'
        WHEN CUSTOMER_TYPE = 3 THEN 'Trader'
        WHEN CUSTOMER_TYPE = 4 THEN 'Private'
        ELSE 'Unknown'
    END AS CUSTOMER_TYPE,
    COUNT(*) AS LowQualityWoodOrders
FROM dbo.WOODCORP_ETC_OFFLINE AS O
JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
ON O.CUSTOMER_NUMBER_OFFLINE = C.CUSTOMER_NO
WHERE LOW_QUALITY_WOOD > 0
GROUP BY CUSTOMER_TYPE
ORDER BY LowQualityWoodOrders DESC;



(5 rows affected)

Total execution time: 00:00:00.063

CUSTOMER_TYPE,LowQualityWoodOrders
Transporter,8819
Factory,4965
Trader,1467
Private,991
Unknown,453


Transporters order low-quality wood the most frequently offline, followed by factories. This indicates that transporters and factories might be more cost-sensitive or use low-quality wood for purposes that do not require high-quality materials. This insight can help in targeting marketing strategies and product recommendations for these customer types to potentially upsell or cross-sell higher quality products or complementary goods.

### **Complex Questions**

In [35]:
-- 1) How many customers ordered both online and offline?
SELECT COUNT(DISTINCT CUSTOMER_NO) AS CustomersBothOnlineOffline
FROM dbo.WOODCORP_ETC_CUSTOMERDATA AS C
WHERE EXISTS (
    SELECT 1
    FROM dbo.WOODCORP_ETC_ONLINE AS O
    WHERE O.CUSTOMER_NUMBER_ONLINE = C.CUSTOMER_NO 
) 
AND EXISTS (
    SELECT 1
    FROM dbo.WOODCORP_ETC_OFFLINE AS OFL
    WHERE OFL.CUSTOMER_NUMBER_OFFLINE = C.CUSTOMER_NO
);


(1 row affected)

Total execution time: 00:00:00.073

CustomersBothOnlineOffline
1312


Out of the entire customer base, 1,312 customers have placed orders both online and offline. This indicates a significant segment of customers who are comfortable with both purchasing channels, showing their flexibility and higher engagement with the company. By analyzing this group's buying behavior and preferences, the company can tailor marketing strategies to meet their needs better. For example, these customers might appreciate seamless transitions between online and offline experiences, such as online order pickups or in-store returns for online purchases.

In [38]:
-- 2) On average how many more/less items does a man order online?
WITH AverageItemsOrdered AS (
    SELECT GENDER, AVG(NUMBER_ITEMS_ONLINE) AS AvgItems
    FROM dbo.WOODCORP_ETC_CUSTOMERDATA AS C
    JOIN dbo.WOODCORP_ETC_ONLINE AS O
    ON C.CUSTOMER_NO = O.CUSTOMER_NUMBER_ONLINE
    GROUP BY GENDER
)
SELECT 
    (SELECT AvgItems FROM AverageItemsOrdered WHERE GENDER = 0) - 
    (SELECT AvgItems FROM AverageItemsOrdered WHERE GENDER = 1) AS DifferenceInAverageItems;


(1 row affected)

Total execution time: 00:00:00.062

DifferenceInAverageItems
1


In [6]:
-- Check difference with offline
WITH AverageItemsOrdered AS (
    SELECT GENDER, AVG(NUMBER_ITEMS_OFFLINE) AS AvgItems
    FROM dbo.WOODCORP_ETC_CUSTOMERDATA AS C
    JOIN dbo.WOODCORP_ETC_OFFLINE AS O
    ON C.CUSTOMER_NO = O.CUSTOMER_NUMBER_OFFLINE
    GROUP BY GENDER
)
SELECT 
    (SELECT AvgItems FROM AverageItemsOrdered WHERE GENDER = 0) - 
    (SELECT AvgItems FROM AverageItemsOrdered WHERE GENDER = 1) AS DifferenceInAverageItems;

(1 row affected)

Total execution time: 00:00:00.088

DifferenceInAverageItems
3


Men tend to order 1 more item online and 3 more items offline compared to women. This could be indicative of wood production being a male-dominated industry, where men are more frequently responsible for placing larger orders. To address this gender-based difference, strategies should be developed to engage female customers more effectively. This could include targeted marketing campaigns that highlight the benefits and uses of wood products in areas that might appeal more to women, such as home improvement and crafts.

In [40]:
-- 3) What is the average offline order value for transporters which have more than 3 offline orders?
SELECT ROUND(AVG(ORDER_VALUE_OFFLINE), 0) AS AverageOfflineOrderValue
FROM dbo.WOODCORP_ETC_OFFLINE AS OFL
JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
ON OFL.CUSTOMER_NUMBER_OFFLINE = C.CUSTOMER_NO
WHERE C.CUSTOMER_TYPE = 1
AND C.CUSTOMER_NO IN (
    SELECT CUSTOMER_NUMBER_OFFLINE
    FROM dbo.WOODCORP_ETC_OFFLINE
    GROUP BY CUSTOMER_NUMBER_OFFLINE -- since we select customer mummer the counting of these means teh numer of orders 
    HAVING COUNT(*) > 3 -- The HAVING clause in SQL is used to filter records that are returned by a GROUP BY clause. It is similar to the WHERE clause, but HAVING is used for aggregate functions like COUNT, SUM, AVG, MIN, MAX, etc., while WHERE is used for filtering rows before the grouping occurs. here it ensures that only transporters who have placed more than three offline orders are considered in the calculation of the average offline order value.
);


(1 row affected)

Total execution time: 00:00:00.094

AverageOfflineOrderValue
4116


In [5]:
-- Check for online
SELECT ROUND(AVG(ORDER_VALUE_ONLINE), 0) AS AverageOnlineOrderValue
FROM dbo.WOODCORP_ETC_ONLINE AS OFL
JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
ON OFL.CUSTOMER_NUMBER_ONLINE = C.CUSTOMER_NO
WHERE C.CUSTOMER_TYPE = 1
AND C.CUSTOMER_NO IN (
    SELECT CUSTOMER_NUMBER_ONLINE
    FROM dbo.WOODCORP_ETC_ONLINE
    GROUP BY CUSTOMER_NUMBER_ONLINE -- since we select customer mummer the counting of these means teh numer of orders 
    HAVING COUNT(*) > 3
);

(1 row affected)

Total execution time: 00:00:00.054

AverageOnlineOrderValue
16596


Transporters with more than 3 orders spend significantly more on online orders (average value of 16,596) compared to offline orders (average value of 4,116). This suggests that transporters are more inclined to place larger orders online. To capitalize on this trend, businesses should focus on strengthening their online sales platform, providing targeted promotions and discounts for online bulk purchases, and improving the overall online purchasing experience for transporters.

In [41]:
-- 4) How many transporters order on average more than 2 discounted items online?
SELECT COUNT(*) AS TransportersWithMoreThan2DiscountedItems
FROM (
    SELECT CUSTOMER_NO, AVG(NUMBER_DISCOUNTED_ITEMS_ONLINE) AS AvgDiscountedItems
    FROM dbo.WOODCORP_ETC_ONLINE AS O
    JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
    ON O.CUSTOMER_NUMBER_ONLINE = C.CUSTOMER_NO
    WHERE C.CUSTOMER_TYPE = 1
    GROUP BY CUSTOMER_NO
    HAVING AVG(NUMBER_DISCOUNTED_ITEMS_ONLINE) > 2
) AS Transporters;


(1 row affected)

Total execution time: 00:00:00.055

TransportersWithMoreThan2DiscountedItems
763


In [14]:
-- CHeck for offline
SELECT COUNT(*) AS TransportersWithMoreThan2DiscountedItems
FROM (
    SELECT CUSTOMER_NO, AVG(NUMBER_DISCOUNT_ITEMS_OFFLINE) AS AvgDiscountedItems
    FROM dbo.WOODCORP_ETC_OFFLINE AS O
    JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
    ON O.CUSTOMER_NUMBER_OFFLINE = C.CUSTOMER_NO
    WHERE C.CUSTOMER_TYPE = 1
    GROUP BY CUSTOMER_NO
    HAVING AVG(NUMBER_DISCOUNT_ITEMS_OFFLINE) > 2
) AS Transporters;


(1 row affected)

Total execution time: 00:00:00.168

TransportersWithMoreThan2DiscountedItems
4049


Transporters show a higher tendency to order more than 2 discounted items offline (4,049) compared to online (763). This indicates that transporters are more responsive to discounts in offline transactions. To leverage this behavior, it is crucial to implement targeted discount campaigns for transporters in the offline market while exploring ways to replicate this success in online platforms, potentially through personalized discount offers and promotions.

In [43]:
-- 5) What age group has the highest online discount average for orders with more than 5 items?
SELECT TOP 1 AgeGroup, ROUND(AVG(DISCOUNT_VALUE_ONLINE), 0) AS AverageDiscount
FROM (
    SELECT CASE
        WHEN AGE = 1 THEN '0-18'
        WHEN AGE = 2 THEN '19-35'
        WHEN AGE = 3 THEN '36-55'
        WHEN AGE = 4 THEN '55+'
        WHEN AGE = 0 THEN 'NA' END AS AgeGroup,
        DISCOUNT_VALUE_ONLINE
    FROM dbo.WOODCORP_ETC_CUSTOMERDATA AS C
    JOIN dbo.WOODCORP_ETC_ONLINE AS O
    ON C.CUSTOMER_NO = O.CUSTOMER_NUMBER_ONLINE
    WHERE NUMBER_ITEMS_ONLINE > 5
) AS AgeDiscounts
GROUP BY AgeGroup
ORDER BY AverageDiscount DESC;


(1 row affected)

Total execution time: 00:00:00.061

AgeGroup,AverageDiscount
36-55,2220


The age group 36-55 has the highest average online discount for orders with more than 5 items, at 2220. This suggests that this age group is particularly responsive to discounts, indicating an opportunity to tailor discount-focused marketing strategies towards them to maximize sales.

## **3) Additional Insights**

### **Additional Insight 1: Frequency of High-Quality Wood Orders Among Different Customer Types**

In [15]:
-- Determine which customer type orders high-quality wood most frequently offline
SELECT 
    CASE 
        WHEN C.CUSTOMER_TYPE = 1 THEN 'Transporter'
        WHEN C.CUSTOMER_TYPE = 2 THEN 'Factory'
        WHEN C.CUSTOMER_TYPE = 3 THEN 'Trader'
        WHEN C.CUSTOMER_TYPE = 4 THEN 'Private'
        ELSE 'Unknown'
    END AS CUSTOMER_TYPE,
    COUNT(CASE WHEN HIGH_QUALITY_WOOD > 0 THEN 1 END) AS HighQualityWoodOrders,
    COUNT(CASE WHEN LOW_QUALITY_WOOD > 0 THEN 1 END) AS LowQualityWoodOrders,
    COUNT(*) AS TotalWoodOrders,
    ROUND(COUNT(CASE WHEN HIGH_QUALITY_WOOD > 0 THEN 1 END) * 100.0 / COUNT(*), 2) AS HighQualityWoodProportion
FROM dbo.WOODCORP_ETC_OFFLINE AS O
JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
ON O.CUSTOMER_NUMBER_OFFLINE = C.CUSTOMER_NO
GROUP BY C.CUSTOMER_TYPE
ORDER BY HighQualityWoodOrders DESC;


Warning: Null value is eliminated by an aggregate or other SET operation.

(5 rows affected)

Total execution time: 00:00:00.090

CUSTOMER_TYPE,HighQualityWoodOrders,LowQualityWoodOrders,TotalWoodOrders,HighQualityWoodProportion
Transporter,7798,8819,39941,19.520000000000
Factory,4580,4965,23539,19.460000000000
Trader,1485,1467,7411,20.040000000000
Private,1091,991,4873,22.390000000000
Unknown,414,453,1805,22.940000000000


Transporters have the highest frequency of high-quality wood orders (7,780), followed by factories (4,580). However, the proportion of high-quality wood orders relative to total orders is highest among private customers (22.34%), traders (20.06%), and factories (19.66%). This indicates that while transporters order the most high-quality wood in absolute terms, private customers and traders have a higher preference for high-quality wood relative to their total orders.

### **Additional Insight 2: Analysis of Discount Utilization by Customer Type**

In [16]:
-- Determine which customer type utilizes discounts the most in online orders
SELECT 
    CASE 
        WHEN CUSTOMER_TYPE = 1 THEN 'Transporter'
        WHEN CUSTOMER_TYPE = 2 THEN 'Factory'
        WHEN CUSTOMER_TYPE = 3 THEN 'Trader'
        WHEN CUSTOMER_TYPE = 4 THEN 'Private'
        ELSE 'Unknown'
    END AS CUSTOMER_TYPE,
    SUM(DISCOUNT_VALUE_ONLINE) AS TotalDiscountValue,
    SUM(ORDER_VALUE_ONLINE) AS TotalOrderValue,
    ROUND(SUM(DISCOUNT_VALUE_ONLINE) * 100.0 / SUM(ORDER_VALUE_ONLINE), 2) AS DiscountProportion
FROM dbo.WOODCORP_ETC_ONLINE AS O
JOIN dbo.WOODCORP_ETC_CUSTOMERDATA AS C
ON O.CUSTOMER_NUMBER_ONLINE = C.CUSTOMER_NO
GROUP BY CUSTOMER_TYPE
ORDER BY TotalDiscountValue DESC;


(5 rows affected)

Total execution time: 00:00:00.049

CUSTOMER_TYPE,TotalDiscountValue,TotalOrderValue,DiscountProportion
Transporter,"6278844,410000007","48577951,86000014","12,93"
Factory,"5936052,78000002","49926713,45","11,89"
Trader,"1642637,8199999987","15086520,67999999","10,89"
Private,"559840,2899999997","5768434,3500000015","9,71"
Unknown,"158809,99000000002","1638136,6600000001","9,69"


Transporters also lead in total discount value utilized in online orders (62,788,444), with factories and traders following. Transporters benefit the most from discounts (12.93% of their total order value), suggesting they are highly sensitive to price reductions.

Conclusion:

Given that transporters have the highest volume of high-quality wood orders and also benefit significantly from discounts, a combined strategy can be effective. Offering discounts on low-quality wood to transporters who frequently order high-quality wood can balance inventory and increase overall sales. Additionally, targeting private customers and traders with promotions for high-quality wood can leverage their higher relative interest in these products, while also offering them discounts on other products to encourage bulk purchasing. This dual approach can optimize customer satisfaction and sales performance across different customer segments.